In [ ]:
import os
os.getcwd()

In [ ]:
# !pip install nsepy pandas numpy matplotlib treeinterpreter sklearn seaborn 

In [ ]:
# Get 5 yrs data from NSE

from nsepy import get_history
from datetime import date
#data = get_history(symbol="NIFTY", start=date(2015,1,1), end=date(2020,5,23), index=True)
data = get_history(symbol="NIFTY", start=date(2015,1,1), end=date(2020,1,1), index=True)

In [ ]:
# Assuming you already have panda imported
df=pd.DataFrame(data)
df.to_csv('nifty_price_data.csv')

In [ ]:
data.describe

In [ ]:
# Convert data into panda datafram

df = pd.DataFrame(data)
X = df.values
X.shape
df.isnull().sum()
df.describe()
df.head()


In [ ]:
# Setup env for running ML

import pandas as pd
import numpy as np
import talib
from talib.abstract import *

import matplotlib as mpl
import matplotlib.pyplot as plt
from treeinterpreter import treeinterpreter as ti

mpl.style.use('ggplot')
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

from collections import Counter, OrderedDict, defaultdict
import operator
from pprint import pprint as pp

%matplotlib inline
np.random.seed(42)
pd.options.display.max_columns = None

In [ ]:
# checkout datafram 

df.head()

In [ ]:
# Check there are no Nulls

df.isnull().sum()

In [ ]:
# Drop columns not needed

df.drop(['Turnover'], axis=1, inplace=True)
df.head()

In [ ]:
# Plot a line graph of data

for f in ['Close',]:
    plt.figure(figsize=(60,20))
    mpl.rcParams.update({'font.size': 25})
    plt.plot(getattr(df, f))
    plt.ylabel(f)
    plt.show()


In [ ]:
# Get functions from talib

analysis = talib.get_function_groups()
# print function names
analysis

In [ ]:
# Run analysis on 5 yrs data

inputs = {
    'open': df.Open,
    'high': df.High,
    'low': df.Low,
    'close': df.Close,
    'volume': df.Volume,
}

# + analysis['Momentum Indicators']
for f in analysis['Volatility Indicators'] + analysis['Statistic Functions'] + analysis['Price Transform']:
    print(f)
    output = getattr(talib.abstract, f)(inputs)
    if isinstance(output, np.ndarray):
        print(f)
        df[f] = output
    else:
        for i in range(len(output)):
            f_i = '%s_%s'%(f, i)
            print(f_i)
            df[f_i] = output[i]

In [ ]:
# Check what data has been populated
df.head()

In [ ]:
# Check data 
print(len(df))
print(len(df.columns))

In [ ]:
# Check last 30 days return
# Create a new column Returns_30day
#df['Returns_30day'] = df.Close.shift(30)/df.Close*100 -100
df['Returns_30day'] = df.Close.shift(15)/df.Close*100 -100
df['Returns_30day'].describe()

In [ ]:
# Plot 30 day rolling return chart

fig, ax1 = plt.subplots()
plt.figure(figsize=(80,30))

mpl.rcParams.update({'font.size': 15})
ax2 = ax1.twinx()

x = range(len(df))
ax1.plot(x, df.Returns_30day, 'g-')
ax2.plot(x, df.Close, 'b-')

ax1.set_xlabel('Time')
ax1.set_ylabel('Returns_30day', color='g')
ax2.set_ylabel('Close', color='b')

plt.show()

In [ ]:
# Define class to find 5% or more returns

def assign_class(x):
    if x>=5:
        return 5
    elif x<=-5:
        return -5
    else:
        return np.trunc(x)
    
assign_class(-0.1)

In [ ]:
# Filter out 5% returns from 30 day rolling
df['Returns_30day'] = df.Returns_30day.apply(assign_class)

In [ ]:
# This prepares data for creating training data
# Dropping unnecessary variables
df.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis=1, inplace=True)

# Dropping created variables
df.dropna(inplace=True)
df.to_csv('nifty_train_data.csv', index=False)
print('saved')

In [ ]:
df.head()